# Data wrangling for Risk factors datasets

In [1]:
#loading library
library(tidyverse)
library(sf)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Linking to GEOS 3.12.0, GDAL 3.7.1, PROJ 9.3.0; sf_use_s2() is TRUE



## 1. Humanities Datasets

### 1.1. New Zealand Health Survey data

In [2]:
#### New Zealand Health Survey data ####
nzhs <- read_csv('data/nz-health-survey-2017-20-regional-update-dhb-prevalences.csv')
nzhs %>% 
  filter(population == "adults") %>% #select adults population group for analysis
  filter(!grepl("-",year) & type == "STD") %>% #filter age-standardized data with type == "STD", remove combined year data.
  mutate(DHB = case_when(region == 'Tairāwhiti' ~ "Tairawhiti", #modify DHB names in DHB_mapto match cancer
                         region == 'Waitematā' ~ 'Waitemata',
                         region == "New Zealand" ~ 'All New Zealand',
                         TRUE ~ region),
         year = as.numeric(str_extract(year,"^[0-9]*")), #just keep the year information
         rf = short.description,
         sex = case_when(sex == 'All' ~ "AllSex",
                         TRUE ~ sex)
  ) %>% 
  select(DHB,year,sex,rf,Prevalence) %>%
  pivot_wider(names_from = c(rf), names_sep = '-',values_from = Prevalence) -> nzhs_wide 


colnames(nzhs_wide)[-c(1,2,3)] <- paste0('NZHS-',colnames(nzhs_wide)[-c(1,2,3)] ) #add NZHS as identify of the data source

write_csv(nzhs_wide,"data/clean/nzhs_wide.csv") #save clean nzhs data for correlation analaysis

nzhs_wide



Rows: 772978 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): population, short.description, region, type, year, agegroup, sex, E...
dbl (5): Prevalence, CL.Lower.Bound, CL.Upper.Bound, estimated.number, sampl...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


DHB,year,sex,NZHS-Physically active,NZHS-After-hours medical centre visit,NZHS-Past-year drinkers,NZHS-Amphetamine use (total population),NZHS-Amphetamine use (16-64 years),NZHS-Anxiety disorder,NZHS-Arthritis,⋯,NZHS-Unmet need for after-hours due to cost,NZHS-Unmet need for after-hours due to lack of transport,NZHS-Unmet need for GP due to cost,NZHS-Unmet need for GP due to lack of transport,NZHS-Unmet need for primary health care,NZHS-Vegetable and fruit intake,NZHS-Vegetable intake,NZHS-Mean waist (cm),NZHS-Mean weight (kg),NZHS-Waist to height ratio ≥ 0.5
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Auckland,2011,AllSex,46.2,11.5,76.8,0.7,0.8,4.7,10.8,⋯,3.3,1.0,8.6,2.7,18.3,37.2,56.3,86.5,75.2,47.4
Auckland,2012,AllSex,51.8,11.3,72.3,0.2,0.2,4.2,9.4,⋯,4.9,0.8,12.6,2.6,21.1,39.1,61.4,88.5,76.2,54.6
Auckland,2013,AllSex,45.4,10.8,74.5,1.2,1.3,6.3,9.2,⋯,3.2,0.5,10.3,2.3,19.4,40.6,56.4,87.7,76.2,55.2
Auckland,2014,AllSex,50.0,12.6,73.7,0.8,0.9,7.0,10.7,⋯,4.4,0.5,11.8,1.7,23.0,41.8,65.4,88.3,76.3,54.7
Auckland,2015,AllSex,44.9,11.6,70.4,1.2,1.4,5.4,11.5,⋯,6.2,1.7,11.3,4.1,25.5,35.4,59.2,89.8,77.1,58.7
Auckland,2016,AllSex,47.0,10.1,74.4,0.9,1.0,6.7,10.0,⋯,6.2,0.7,10.3,2.0,21.5,30.3,46.9,90.3,77.8,61.0
Auckland,2011,Female,43.8,12.8,71.2,0.4,0.4,4.8,10.8,⋯,4.9,1.0,11.4,4.1,22.6,41.2,60.8,81.6,68.9,40.9
Auckland,2011,Male,47.9,9.6,82.4,1.1,1.3,4.7,11.1,⋯,1.6,0.7,5.3,0.8,13.4,31.6,50.6,92.0,82.4,53.8
Auckland,2012,Female,46.7,14.0,66.7,0.0,0.0,5.7,10.7,⋯,7.6,1.6,18.1,4.1,27.2,40.8,62.9,84.1,70.2,48.5


### 1.2. Education data

In [3]:
#### Education data ####

education <- read.csv('data/people/Statistical Area 1 dataset for Census 2018 – total New Zealand – Long format_updated_16-7-20/Individual_part2_totalNZ_updated_16-7-20/Highest_qualification_long_updated_16-7-20.csv')

#clean the Education data and map to qualification levels
qualifications_to_level <- c(
  "No qualification" = 'No qualification',
  "Level 1 certificate" = 'level 1',
  "Level 2 certificate" = 'level 2',
  "Level 3 certificate" = 'level 3',
  "Level 4 certificate" = 'level 4',
  "Level 5 diploma" = 'level 5',
  "Level 6 diploma" = 'level 6',
  "Bachelor degree and Level 7 qualification" = 'level 7',
  "Post-graduate and honours degrees" = 'level 8',
  "Masters degree" = 'level 9',
  "Doctorate degree" = 'level 10'
)

#create new variables with the "≥ levels" format
qualifications_map <- c(
  'No qualification' = "≥ level 1",
  'level 1' = "≥ level 2",
  'level 2' = "≥ level 3",
  'level 3' = "≥ level 4",
  'level 4' = "≥ level 5",
  'level 5' = "≥ level 6",
  'level 6' = "≥ level 7",
  'level 7' = "≥ level 8",
  'level 8' = "≥ level 9",
  'level 9' = "≥ level 10"
)


education %>% 
  mutate(Area_description = str_replace( Area_description,'Capital and Coast',"Capital & Coast")) %>% #unify DHB names
  filter(Year >= 2011 & Year <= 2020 & #screen years match the cancer data
           grepl("DHB",Area_code_and_description) & #filter rows with DHB information
           !Highest_qualification_descriptor %in% c('Overseas secondary school qualification', "Not elsewhere included","Total")) %>% #keep only NZ education qualication data
  rename( DHB = Area_description, year = Year) %>%
  mutate(highest_edu =  qualifications_to_level[Highest_qualification_descriptor], Count = as.numeric(Count) ) %>%
  group_by(DHB,year) %>%
  mutate(percentage = Count / Count[Highest_qualification_descriptor == "Total stated"] * 100) %>% #calculate the percentage of each qualification level
  filter(Highest_qualification_descriptor != 'Total stated') %>% 
  select(DHB,highest_edu,percentage,year) -> edu1

edu1 %>%  
  group_by(DHB,year) %>%
  mutate(percentage = 100 - cumsum(percentage)) %>% #calculate the cumulative percentage of each "≥ level" variable
  filter(highest_edu != 'level 10') %>% #remove the last variable which has the 100% percentage 
  mutate(highest_edu = qualifications_map[highest_edu]) -> edu2

edu <- rbind(edu1,edu2) %>% pivot_wider(names_from = highest_edu,values_from = percentage)

colnames(edu)[-c(1,2)] <- paste0('Education-',colnames(edu)[-c(1,2)] )


write_csv(edu,"data/clean/highest_qulification.csv")
edu


DHB,year,Education-No qualification,Education-level 1,Education-level 2,Education-level 3,Education-level 4,Education-level 5,Education-level 6,Education-level 7,⋯,Education-≥ level 1,Education-≥ level 2,Education-≥ level 3,Education-≥ level 4,Education-≥ level 5,Education-≥ level 6,Education-≥ level 7,Education-≥ level 8,Education-≥ level 9,Education-≥ level 10
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Northland,2013,27.40435,16.615632,11.193031,7.016083,11.243571,4.040194,5.238279,8.767131,⋯,72.59565,55.98002,44.78699,37.77091,26.52734,22.48714,17.24886,8.481731,6.647442,5.187740
Waitemata,2013,15.81787,11.568058,10.413008,10.246176,9.079951,4.810179,4.901178,15.885724,⋯,84.18213,72.61407,62.20106,51.95488,42.87493,38.06475,33.16357,17.277851,14.219004,10.885565
Auckland,2013,12.17040,7.818630,8.193017,11.340323,5.810557,4.663761,4.676997,23.289497,⋯,87.82960,80.01097,71.81795,60.47763,54.66707,50.00331,45.32631,22.036815,17.421270,11.762926
Counties Manukau,2013,23.12173,12.265550,10.140840,9.942133,8.937936,4.596431,4.025512,11.698508,⋯,76.87827,64.61272,54.47188,44.52974,35.59181,30.99538,26.96986,15.271356,13.363769,11.279770
Waikato,2013,24.84819,14.594582,11.163887,8.790178,10.646818,4.087252,4.713748,11.351475,⋯,75.15181,60.55723,49.39335,40.60317,29.95635,25.86910,21.15535,9.803874,7.550414,5.446062
Lakes,2013,24.85365,15.662595,11.435815,7.951141,11.993547,4.448029,5.079511,9.859415,⋯,75.14635,59.48375,48.04794,40.09680,28.10325,23.65522,18.57571,8.716294,6.715833,5.107168
Bay of Plenty,2013,24.36634,15.392363,11.244230,7.557700,11.707931,4.609736,5.784725,10.266471,⋯,75.63366,60.24129,48.99706,41.43936,29.73143,25.12170,19.33697,9.070499,7.049937,5.564415
Tairawhiti,2013,28.34299,16.172998,12.459755,8.338699,10.420691,3.713243,5.011805,9.476282,⋯,71.65701,55.48401,43.02425,34.68555,24.26486,20.55162,15.53982,6.063533,4.185448,3.069328
Taranaki,2013,28.22619,16.312589,11.396304,7.182909,12.186069,4.043595,4.762281,8.963829,⋯,71.77381,55.46122,44.06492,36.88201,24.69594,20.65235,15.89006,6.926236,5.149266,3.842205


### 1.3. Working hours data

In [4]:

#### Working hours ####

work_hours <- read.csv('data/people/Statistical Area 1 dataset for Census 2018 – total New Zealand – Long format_updated_16-7-20/Individual_part3b_totalNZ_updated_16-7-20/total_hours_worked_long_updated_16-7-20.csv')

#create new variables with the "≥ hours" format
work_hours_map <- c(
  "1-9 hours worked" = "≥ 10 hours"  ,
  "10-19 hours worked" = "≥ 20 hours",
  "20-29 hours worked" = "≥ 30 hours",
  "30-39 hours worked" = "≥ 40 hours",
  "40-49 hours worked" = "≥ 50 hours",
  "50-59 hours worked" = "≥ 60 hours"
)

work_hours %>% 
  mutate(Area_description = str_replace( Area_description,'Capital and Coast',"Capital & Coast")) %>% #unify DHB names
  filter(Year >= 2011 & Year <= 2020 &  #screen years match the cancer data
           grepl("DHB",Area_code_and_description) & #filter rows with DHB information
           !Hours_worked_week_descriptor %in% c( "Not elsewhere included","Total")) %>%
  rename( DHB = Area_description, year = Year) %>%
  mutate(Count = as.numeric(Count) ) %>%
  group_by(DHB,year) %>%
  mutate(percentage = Count / Count[Hours_worked_week_descriptor == "Total stated"] * 100) %>% #calculate the percentage of each work hours length
  filter(Hours_worked_week_descriptor != 'Total stated') %>% 
  select(DHB,Hours_worked_week_descriptor,percentage,year) -> whs1

whs1 %>%  
  group_by(DHB,year) %>%
  mutate(percentage = 100 - cumsum(percentage)) %>% #calculate the cumulative percentage of each "≥ hours" variable
  filter(Hours_worked_week_descriptor != '60 hours or more worked') %>% #remove the last variable which has the 100% percentage 
  mutate(Hours_worked_week_descriptor = work_hours_map[Hours_worked_week_descriptor]) -> whs2

whs <- rbind(whs1,whs2) %>% pivot_wider(names_from = Hours_worked_week_descriptor,values_from = percentage)

colnames(whs)[-c(1,2)] <- paste0('WorkHours-',colnames(whs)[-c(1,2)] )


write_csv(whs,"data/clean/work_hours.csv")

whs

DHB,year,WorkHours-1-9 hours worked,WorkHours-10-19 hours worked,WorkHours-20-29 hours worked,WorkHours-30-39 hours worked,WorkHours-40-49 hours worked,WorkHours-50-59 hours worked,WorkHours-60 hours or more worked,WorkHours-≥ 10 hours,WorkHours-≥ 20 hours,WorkHours-≥ 30 hours,WorkHours-≥ 40 hours,WorkHours-≥ 50 hours,WorkHours-≥ 60 hours
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Northland,2013,5.855106,8.145103,10.830644,15.88946,36.91579,11.247007,11.122098,94.14489,85.99979,75.16915,59.27969,22.36390,11.116894
Waitemata,2013,5.346188,7.234256,9.181169,14.17428,46.52060,10.917741,6.625767,94.65381,87.41956,78.23839,64.06410,17.54351,6.625767
Auckland,2013,4.862268,6.535578,8.692289,13.60662,47.07431,12.157901,7.074012,95.13773,88.60215,79.90986,66.30325,19.22894,7.071038
Counties Manukau,2013,4.944966,6.053739,8.276141,12.41178,50.87731,10.553577,6.884105,95.05503,89.00129,80.72515,68.31337,17.43606,6.882486
Waikato,2013,5.646658,7.667285,9.659211,13.89947,40.67660,11.528673,10.925929,94.35334,86.68606,77.02685,63.12738,22.45078,10.922103
Lakes,2013,5.497194,7.626997,10.231688,14.72154,39.97698,11.454886,10.490718,94.50281,86.87581,76.64412,61.92258,21.94560,10.490718
Bay of Plenty,2013,5.623772,7.977828,11.163345,15.73463,39.60848,11.159837,8.739124,94.37623,86.39840,75.23505,59.50042,19.89194,8.732108
Tairawhiti,2013,5.914548,7.379944,10.187147,15.18362,39.17726,11.705508,10.451977,94.08545,86.70551,76.51836,61.33475,22.15749,10.451977
Taranaki,2013,5.551554,7.322050,10.064818,13.55179,41.26155,10.821030,11.415196,94.44845,87.12640,77.06158,63.50978,22.24823,11.427200


### 1.4. Income data 

In [5]:
#### 2.4.3 Income data  ####

income_cesus <- read.csv('data/people/Statistical Area 1 dataset for Census 2018 – total New Zealand – Long format_updated_16-7-20/Individual_part2_totalNZ_updated_16-7-20/Total_personal_income_long_updated_16-7-20.csv')

#create new variables with the "≥ $" format
income_map <- c(
  "$5,000 or less" = "> $5,000",
  "$5,001-$10,000" = "> $10,000",
  "$10,001-$20,000" = "> $20,000",
  "$20,001-$30,000" = "> $30,000",
  "$30,001-$50,000" = "> $50,000",
  "$50,001-$70,000" = "> $70,000"
)

income_cesus %>% 
  mutate(Area_description = str_replace( Area_description,'Capital and Coast',"Capital & Coast")) %>% #unify DHB names
  filter(Year >= 2011 & Year <= 2020 &  #screen years match the cancer data
           grepl("DHB",Area_code_and_description) & #filter rows with DHB information
           !Grouped_personal_income_descriptor %in% c( "Not stated","Total","Median, ($)")) %>%
  rename( DHB = Area_description, year = Year) %>%
  mutate(Count = as.numeric(Count) ) %>%
  group_by(DHB,year) %>%
  mutate(percentage = Count / Count[Grouped_personal_income_descriptor == "Total stated"] * 100) %>% #calculate the percentage of each income range
  filter(Grouped_personal_income_descriptor != 'Total stated') %>% 
  select(DHB,Grouped_personal_income_descriptor,percentage,year) -> income1


income1 %>%  
  group_by(DHB,year) %>%
  mutate(percentage = 100 - cumsum(percentage)) %>% #calculate the cumulative percentage of each "≥ $" variable
  filter(Grouped_personal_income_descriptor != '$70,001 or more') %>% #remove the last variable which has the 100% percentage 
  mutate(Grouped_personal_income_descriptor = income_map[Grouped_personal_income_descriptor]) -> income2


income <- rbind(income1,income2) %>% pivot_wider(names_from = Grouped_personal_income_descriptor,values_from = percentage)

colnames(income)[-c(1,2)] <- paste0('Income-',colnames(income)[-c(1,2)] )


write_csv(income,"data/clean/income.csv")

income

DHB,year,"Income-$5,000 or less","Income-$5,001-$10,000","Income-$10,001-$20,000","Income-$20,001-$30,000","Income-$30,001-$50,000","Income-$50,001-$70,000","Income-$70,001 or more","Income-> $5,000","Income-> $10,000","Income-> $20,000","Income-> $30,000","Income-> $50,000","Income-> $70,000"
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Northland,2013,13.532017,5.814834,24.23819,16.448173,20.34901,10.71200,8.905786,86.46798,80.65315,56.41496,39.96679,19.61778,8.905786
Waitemata,2013,16.524257,5.107813,15.67295,12.109951,20.44861,13.99165,16.143972,83.47574,78.36793,62.69498,50.58502,30.13641,16.144761
Auckland,2013,18.259650,5.768763,14.23311,10.361716,18.86999,12.91429,19.591551,81.74035,75.97159,61.73848,51.37676,32.50678,19.592485
Counties Manukau,2013,19.503050,5.738617,16.26357,12.128390,21.65887,12.89463,11.810963,80.49695,74.75833,58.49476,46.36637,24.70751,11.812881
Waikato,2013,13.969606,5.379930,19.66649,14.200225,21.95494,12.68642,12.142393,86.03039,80.65046,60.98397,46.78375,24.82881,12.142393
Lakes,2013,13.163367,5.135160,19.73601,15.730947,22.97713,12.50339,10.749480,86.83663,81.70147,61.96546,46.23452,23.25739,10.754001
Bay of Plenty,2013,12.544604,5.219228,21.55367,16.359050,21.51265,11.83709,10.975760,87.45540,82.23617,60.68250,44.32345,22.81080,10.973709
Tairawhiti,2013,13.831678,6.001045,22.02823,16.623105,21.89232,11.28071,8.332462,86.16832,80.16728,58.13905,41.51594,19.62363,8.342917
Taranaki,2013,12.127397,4.694601,19.52168,14.931064,22.34460,12.99006,13.390588,87.87260,83.17800,63.65632,48.72526,26.38065,13.390588


### 1.5. Birth numbers

In [6]:
#### Birth numbers ####

children_census <- read.csv('data/people/Statistical Area 1 dataset for Census 2018 – total New Zealand – Long format_updated_16-7-20/Individual_part2_totalNZ_updated_16-7-20/Number_of_children_born_long_updated_16-7-20.csv')

#create new variables with the "> children" format
children_map <- c(
    "No children" = "> 0 children",
    "One child" = "> 1 children",
    "Two children" = "> 2 children",
    "Three children" = "> 3 children",
    "Four children" = "> 4 children",
    "Five children" = "> 5 children"
  )


children_census %>% 
  mutate(Area_description = str_replace( Area_description,'Capital and Coast',"Capital & Coast")) %>% #unify DHB names
  filter(Year >= 2011 & Year <= 2020 & #screen years match the cancer data
           grepl("DHB",Area_code_and_description) & #filter rows with DHB information
           !Number_children_born_descriptor %in% c("Total","Not elsewhere included","Object to answering")) %>%
  rename( DHB = Area_description, year = Year) %>%
  mutate(Count = as.numeric(Count) ) %>%
  group_by(DHB,year) %>%
  mutate(percentage = Count / Count[Number_children_born_descriptor == "Total stated"] * 100) %>%  #calculate the percentage 
  filter(Number_children_born_descriptor != 'Total stated') %>% 
  select(DHB,Number_children_born_descriptor,percentage,year) -> children1


children1 %>%  
  group_by(DHB,year) %>%
  mutate(percentage = 100 - cumsum(percentage)) %>% #calculate the cumulative percentage of each "> children" variable
  filter(Number_children_born_descriptor != 'Six or more children') %>%  #remove the last variable which has the 100% percentage 
  mutate(Number_children_born_descriptor = children_map[Number_children_born_descriptor]) -> children2

children <- rbind(children1,children2) %>% pivot_wider(names_from = Number_children_born_descriptor,values_from = percentage)

colnames(children)[-c(1,2)] <- paste0('Children-',colnames(children)[-c(1,2)] )



write_csv(children,"data/clean/number_of_children.csv")

children

DHB,year,Children-No children,Children-One child,Children-Two children,Children-Three children,Children-Four children,Children-Five children,Children-Six or more children,Children-> 0 children,Children-> 1 children,Children-> 2 children,Children-> 3 children,Children-> 4 children,Children-> 5 children
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Northland,2013,22.12185,11.16309,26.58493,18.841749,9.727375,3.968382,3.833952,77.87815,66.71506,40.130129,21.288380,11.561004,7.592622
Waitemata,2013,30.88502,14.25600,28.51496,14.890487,5.661387,1.925763,1.405688,69.11498,54.85899,26.344022,11.453535,5.792148,3.866385
Auckland,2013,43.15818,13.86859,22.20325,10.961145,4.278474,1.691118,1.543523,56.84182,42.97324,20.769983,9.808838,5.530364,3.839246
Counties Manukau,2013,29.47177,13.40907,25.35578,14.803652,7.227949,3.359846,3.539965,70.52823,57.11916,31.763384,16.959732,9.731783,6.371937
Waikato,2013,27.20860,11.25583,25.16088,18.170761,8.805643,3.458570,2.907941,72.79140,61.53557,36.374693,18.203932,9.398288,5.939718
Lakes,2013,24.42435,12.45081,25.94825,18.102654,9.093193,3.608809,3.148288,75.57565,63.12484,37.176589,19.073935,9.980742,6.371933
Bay of Plenty,2013,22.86990,11.40089,27.93066,19.448124,9.201711,3.482342,2.668534,77.13010,65.72921,37.798554,18.350430,9.148719,5.666376
Tairawhiti,2013,22.74286,11.63810,23.73333,18.742857,10.400000,4.419048,4.819048,77.25714,65.61905,41.885714,23.142857,12.742857,8.323810
Taranaki,2013,23.39533,11.14408,25.87018,19.842246,9.979014,3.762935,2.822201,76.60467,65.46060,39.590419,19.748173,9.769158,6.006223


## 2. Environment Data

Environment data was mapped to DHB using coordinates data and DHB_map geometry map data

### 2.1 DHB geomtery map data

In [7]:
DHB_map <- st_read("data/NZ_District_Health_Board_boundaries_-_generalised.kml", quiet=TRUE)
DHB_map %>% 
  mutate(DHB = case_when(DHB_name == "Capital and Coast" ~ "Capital & Coast", #modify DHB names in DHB_mapto match cancer
                         TRUE ~ DHB_name)) %>%
  select(DHB,geometry) -> 
  DHB_map

# write_sf(DHB_map,"data/clean/mapdata/DHB_map.shp")

### 2.2. Earthquake data

In [8]:
### Earthquake data ####

earthquake_2007_to_2023 <- read_csv('data/environment/earthquake2007-2023.csv') #load earthquake data

earthquake_2007_to_2023 %>% 
  filter(eventtype == 'earthquake') %>%
  filter(origintime > as.Date("2011-01-01") &  origintime < as.Date("2020-12-31")) %>% #filter year matching cancer data
  mutate(year = format(origintime, "%Y"))%>% #extrat year information 
  select(year,longitude,latitude,magnitude,depth) %>% 
  st_as_sf(.,coords = c("longitude", "latitude"), crs = 4326) %>% #transform into sf object for coordinates mapping
  st_join(.,DHB_map) %>% # coordinates mapping
  filter(!is.na(DHB))-> #remove coordinates that failed to map to DHB region
  earthquake_map

earthquake_map %>%
  as.data.frame() %>%
  select(-geometry) %>%
  group_by(year,DHB) %>% #summarize based on the year and DHB group
  summarise(magnitude_max =  max(magnitude), magnitude_mean = mean(magnitude), 
            depth_max =  max(depth), depth_mean = mean(depth), 
            counts = n()) %>%
  pivot_wider(names_from = DHB, names_sep = "-" , values_from = c(3:7), values_fill = 0) %>% #some DHB does not have earthquake, change to generate those rows with 0
  pivot_longer(names_to = c('category','DHB'), names_sep = '-', values_to = "values", cols = -1) %>% #change back to wide, but all values are in single column
  pivot_wider(names_from = category , values_from = values) -> #change values back to their original category
  earthquake

colnames(earthquake)[-c(1,2)] <- paste0('Earthquake-',colnames(earthquake)[-c(1,2)] )

write_csv(earthquake,"data/clean/earthquake.csv")
earthquake

Rows: 380390 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (8): publicid, eventtype, magnitudetype, depthtype, evaluationmethod, ...
dbl  (11): longitude, latitude, magnitude, depth, usedphasecount, usedstatio...
dttm  (2): origintime, modificationtime

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'year'. You can override using the
`.groups` argument.


year,DHB,Earthquake-magnitude_max,Earthquake-magnitude_mean,Earthquake-depth_max,Earthquake-depth_mean,Earthquake-counts
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2011,Bay of Plenty,4.621000,2.450072,291.50710,63.782210,641
2011,Canterbury,6.200000,2.719806,66.41180,6.929219,5806
2011,Capital & Coast,3.683000,2.341083,66.60430,35.445183,36
2011,Counties Manukau,1.863000,1.743333,12.00000,7.333333,3
2011,Hawke's Bay,4.700000,2.330024,84.66550,35.906967,1508
2011,Hutt Valley,4.000000,2.157663,51.79010,27.061259,169
2011,Lakes,5.600000,2.289090,251.86440,53.840121,1227
2011,MidCentral,4.154000,2.253515,77.10210,32.051992,792
2011,Nelson Marlborough,4.409000,1.938277,226.08310,66.453319,734


### 2.3. Temperature data

In [9]:
### Temperature data ####
#load temperature data with read_xlsx and suppress warnings messages
suppressWarnings(tmp_1951_to_2022 <- readxl::read_xlsx('data/environment/annual&seasonal-temperature 1951-2022.xlsx'))


tmp_1951_to_2022 %>% 
  filter(year >= 2011 & year <= 2020) %>% #filter year matching cancer data
  select( year, site, statistic, season , temperature,lat,lon) %>% 
  st_as_sf(.,coords = c("lon", "lat"), crs = 4326) %>% #transform into sf object for coordinates mapping
  st_join(.,DHB_map) %>% # coordinates mapping
  filter(!is.na(DHB))-> #remove coordinates that failed to map to DHB region
  tmp_map

tmp_map %>%
  as.data.frame() %>%
  select(-c(geometry,site)) %>%
  group_by(year,DHB,statistic,season) %>% #summarize based on the year and DHB group
  summarise(temperature = mean(temperature)) %>%
  pivot_wider(names_from = c(statistic,season), names_sep = '_', values_from = temperature) ->
  tmp

colnames(tmp)[-c(1,2)] <- paste0('Temperature-',colnames(tmp)[-c(1,2)] )


# write_sf(tmp_map,"data/clean/mapdata/temp_map.shp")
write_csv(tmp,"data/clean/temperature.csv")


`summarise()` has grouped output by 'year', 'DHB', 'statistic'. You can
override using the `.groups` argument.


### 2.4. Air quality data

In [10]:
### Air quality data ####
air_2016_to_2022 <- readxl::read_xlsx('data/environment/air-quality2016-2022.xlsx')

air_2016_to_2022 %>% 
  mutate(year =  format(as.Date(`Sample Date`),'%Y'), #extrate year info
         Latitude = as.numeric(Latitude),
         Longitude = as.numeric(Longitude)) %>%
  filter(year >= 2011 & year <= 2020) %>% #filter year matching cancer data
  select( Latitude, Longitude, year, Indicator,Concentration) %>% 
  st_as_sf(.,coords = c("Longitude", "Latitude"), crs = 4326) %>% #transform into sf object for coordinates mapping
  st_join(.,DHB_map) %>%  # coordinates mapping
  filter(!is.na(DHB))-> #remove coordinates that failed to map to DHB region
  air_map

air_map %>%
  as.data.frame() %>%
  select(-c(geometry)) %>%
  group_by(year,DHB,Indicator) %>%  #summarize based on the year and DHB group
  summarise(concentration_max = max(Concentration,na.rm=T),
            concentration_mean = mean(Concentration,na.rm=T)) %>%
  pivot_wider(names_from = c(Indicator), names_sep = '_', values_from = c(concentration_max,concentration_mean)) ->
  air # there are some missing values

colnames(air)[-c(1,2)] <- paste0('Air-',colnames(air)[-c(1,2)] )

write_csv(air,"data/clean/air.csv")
air

`summarise()` has grouped output by 'year', 'DHB'. You can override using the
`.groups` argument.


year,DHB,Air-concentration_max_PM10,Air-concentration_max_PM2.5,Air-concentration_mean_PM10,Air-concentration_mean_PM2.5
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2016,Auckland,62.60000,19.40000,16.03421,6.635103
2016,Bay of Plenty,26.36600,NA,10.19387,NA
2016,Canterbury,96.00800,68.42500,18.32502,8.996711
2016,Capital & Coast,26.23000,12.82000,11.34600,5.500895
2016,Counties Manukau,45.00000,10.00000,13.23914,4.284375
2016,Hawke's Bay,70.63000,68.40000,16.21044,8.124490
2016,Hutt Valley,31.77000,36.11000,10.77229,6.061389
2016,Lakes,56.33300,76.65500,12.99791,9.127861
2016,Nelson Marlborough,65.82000,46.00000,16.28206,9.771443


### 2.5. Ground water quality ###

In [11]:
#### Ground water quality ####

suppressWarnings(water_2014_to_2021 <- readxl::read_xlsx('data/environment/ground water qualitiy 2014-2021.xlsx'))

water_2014_to_2021 %>% 
  mutate(year =  format(as.Date(Date),'%Y')) %>% #extrate year info
  filter(year >= 2011 & year <= 2020) %>% #filter year matching cancer data
  select( Latitude, Longitude, year, Indicator,CensoredValue) %>% 
  st_as_sf(.,coords = c("Longitude", "Latitude"), crs = 4326) %>% #transform into sf object for coordinates mapping
  st_join(.,DHB_map) %>% # coordinates mapping
  filter(!is.na(DHB)) -> #remove coordinates that failed to map to DHB region
  water_map

water_map %>%
  as.data.frame() %>%
  mutate(Indicator = str_replace(Indicator,"E\\. coli","E\\. Coli")) %>%
  select(-c(geometry)) %>%
  group_by(year,DHB,Indicator) %>% #summarize based on the year and DHB group
  summarise(censoredValue_max = max(CensoredValue,na.rm=T),
            censoredValue_mean = mean(CensoredValue,na.rm=T)) %>%
  pivot_wider(names_from = c(Indicator), names_sep = '_', values_from = c(censoredValue_max,censoredValue_mean)) ->
  water # there are some missing values

colnames(water)[-c(1,2)] <- paste0('Water-',colnames(water)[-c(1,2)] )


write_csv(water,"data/clean/water.csv")

`summarise()` has grouped output by 'year', 'DHB'. You can override using the
`.groups` argument.


## 3. Combine Cancer and Risk Factors data

In [12]:
#loading all cealn datasets
#Risk factors datasets
water = read_csv("data/clean/water.csv")
air = read_csv("data/clean/air.csv")
tmp = read_csv("data/clean/temperature.csv")
earthquake = read_csv("data/clean/earthquake.csv")
income = read_csv("data/clean/income.csv")
education = read_csv("data/clean/highest_qulification.csv")
children = read_csv("data/clean/number_of_children.csv")
workhours = read_csv("data/clean/work_hours.csv")
nzhs =  read_csv("data/clean/nzhs_wide.csv")

#Cancer datasets
#use sex filtered incidence and mortality data for analysis, e.g., for breast cancer only include female data and exlcuded allsex data.
incidence = read_csv("data/clean/incidence_sexfiltered.csv")
mortality = read_csv("data/clean/mortality_sexfiltered.csv")

Rows: 200 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): DHB
dbl (11): year, Water-censoredValue_max_Chloride, Water-censoredValue_max_Di...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 88 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): DHB
dbl (5): year, Air-concentration_max_PM10, Air-concentration_max_PM2.5, Air-...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 180 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): DHB
dbl (16): year, Temperature-Average_Annual, Temperature-Average_Autumn, Temp...

ℹ Use `spec()` to retrieve the full column specification

In [13]:
#Combine all rf datasets as a list except NZHS
rf <- list("water" = water, 
           "air" = air,
           "tmp" = tmp,
           "earthquake" = earthquake,
           "income" = income,
           "education" = education,
           "children" = children,
           "workhours" = workhours)

# join cancer incidence data and rf datasets 
incidence %>% 
  filter(DHB != "All New Zealand") %>%
  split(., .[,"cancer"]) %>% #split by cancer
    lapply(., function(df){
      ls = split(df, df$sex) #split by sex
      lapply(ls, function(sub_df){
        sub_df1 = sub_df[, c('year','DHB',"incidence_rate")]
        sub_ls1 = lapply(rf,inner_join, x=sub_df1, by = c("DHB","year")) #join rf and incidence
        names(sub_ls1) = names(rf)
        
        sub_df2 = sub_df[, c('year','DHB',"sex","incidence_rate")] #sex was used for joining NZHS data which has gender information available
        sub_ls2 <- list("nzhs" = inner_join(sub_df2,nzhs,by = c("DHB","year","sex")))
        return(c(sub_ls1,sub_ls2))
        })
      } ) -> incidence_rf

# join cancer mortality data and rf datasets 
mortality %>% 
  filter(DHB != "All New Zealand") %>%
  split(., .[,"cancer"])%>% #split by cancer
  lapply(., function(df){
    ls = split(df, df$sex) #split by sex
    lapply(ls, function(sub_df){
      sub_df1 = sub_df[, c('year','DHB',"mortality_rate")]
      sub_ls1 = lapply(rf,inner_join, x=sub_df1, by = c("DHB","year")) #join rf and mortality
      names(sub_ls1) = names(rf)
      
      sub_df2 = sub_df[, c('year','DHB',"sex","mortality_rate")] #sex was used for joining NZHS data which has gender information available
      sub_ls2 <- list("nzhs" = inner_join(sub_df2,nzhs,by = c("DHB","year","sex")))
      return(c(sub_ls1,sub_ls2))
    })
  } ) -> mortality_rf

save(incidence_rf,file = 'data/clean/incidence_rf.Rdata')
save(mortality_rf, file = 'data/clean/mortality_rf.Rdata')